In [1]:
import pandas as pd
train_df = pd.read_csv('C:/Users/USER/Desktop/Data_Practices/Disaster_Tweeter/train.csv')
test_df = pd.read_csv('C:/Users/USER/Desktop/Data_Practices/Disaster_Tweeter/test.csv')

## 훈련 데이터 탐색

In [2]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
# 키워드에 대한 변수 확인
train_df.keyword.unique()[:10]
# 키워드에 따라 어떤 정보가 주어져있을 가능성이 매우 높다.

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon'], dtype=object)

In [4]:
# target에 대한 정보 확인
train_df.target.value_counts()
# 재난 트윗 3271건, 비재난 트윗 4342건 발생
# 주어진 데이터는 크게 불균형이 발생하지 않았다.

0    4342
1    3271
Name: target, dtype: int64

## 문자열 정제
다음과 같은 정제가 필요할 것이다.  
1. 영어이므로 대소문자를 통일한다.
2. 특수기호 등을 없애줘야 한다.  
3. 의미 없는 문법에 의한 단어들(stopwords)를 제거한다.

이 중 stop_words의 경우에는, tf-idf vectorizer 과정에서 지정가능하므로
나머지 과정에 대해 처리한다.

In [5]:
import re
import string
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) # 괄호 제거
    text = re.sub('https?://\S+|www\.\S+', '', text) # 링크 제거
    text = re.sub('<.*?>+', '', text) 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #구두점 제거
    text = re.sub('\n', '', text) # 
    text = re.sub('\w*\d\w*', '', text)
    return text

In [6]:
train_df.text = train_df.text.apply(lambda s : clean_text(s))
test_df.text = test_df.text.apply(lambda s : clean_text(s))

In [7]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,people receive wildfires evacuation orders in...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,ariaahrary thetawniest the out of control wild...,1
7610,10871,NaN,NaN,s of volcano hawaii,1
7611,10872,NaN,NaN,police investigating after an ebike collided w...,1


In [8]:
train_df = train_df.fillna('.')
test_df = test_df.fillna('.')

## Vectorizer 적용 후의 간단한 분류 모델 평가

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')
train_vec = tfidf.fit_transform(train_df.text)
test_vec = tfidf.transform(test_df.text)

In [10]:
train_vec.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
clf_nb = MultinomialNB()
scores = cross_val_score(clf_nb, train_vec, train_df['target'], cv = 5,
                        scoring = 'f1')
print(scores)

[0.59398496 0.58263773 0.6460251  0.60493827 0.73635665]


## 모델 개선 시도

다음에 대해 고려하고 모델을 적용해보자.  
- 단어의 기본형만 사용할 수 없을 것인가?
- 다른 벡터화는 존재하지 않는가?
- 다른 정보를 사용할 수는 없을 것인가?

### lemmatizer(표제어 추출)
이 과정을 통해 단어를 표제어 기준의 형태로 변경해볼 수 있다.  
하지만 이를 위해서는 위의 형태로 진행하는 것이 아닌 토큰화를 별도로 진행해야 할 것이다.

In [12]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
def text_preprocess(text):
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    tokens = tokenizer.tokenize(text)
    lemmatize_result = " ".join(lemmatizer.lemmatize(token) for token in tokens)
    
    return lemmatize_result

In [14]:
train_df.text = train_df.text.apply(lambda s : text_preprocess(s))
test_df.text = test_df.text.apply(lambda s : text_preprocess(s))

In [15]:
train_df

,id,keyword,location,text,target
0,1,.,.,our deed are the reason of this earthquake may...,1
1,4,.,.,forest fire near la ronge sask canada,1
2,5,.,.,all resident asked to shelter in place are bei...,1
3,6,.,.,people receive wildfire evacuation order in ca...,1
4,7,.,.,just got sent this photo from ruby alaska a sm...,1
...,...,...,...,...,...
7608,10869,.,.,two giant crane holding a bridge collapse into...,1
7609,10870,.,.,ariaahrary thetawniest the out of control wild...,1
7610,10871,.,.,s of volcano hawaii,1
7611,10872,.,.,police investigating after an ebike collided w...,1


In [16]:
# 추가 전처리 후 동일 모델로 재평가
tfidf = TfidfVectorizer(stop_words = 'english')
train_vec = tfidf.fit_transform(train_df.text)
test_vec = tfidf.transform(test_df.text)

clf_nb = MultinomialNB()
scores = cross_val_score(clf_nb, train_vec, train_df['target'], cv = 5,
                        scoring = 'f1')
print(scores)

[0.59128823 0.60855263 0.64585045 0.61672474 0.74294432]


In [17]:
# 다른 vectorizer 활용
from sklearn.feature_extraction.text import CountVectorizer
cnt = CountVectorizer(stop_words = 'english')
train_vec = cnt.fit_transform(train_df.text)
test_vec = cnt.transform(test_df.text)

clf_nb = MultinomialNB()
scores = cross_val_score(clf_nb, train_vec, train_df['target'], cv = 5,
                        scoring = 'f1')
print(scores)

[0.62966031 0.60770328 0.65671642 0.66010598 0.73203492]


count vectorzier에서 더 좋은 성능을 보이고 있는데, 이는 재난 문자에서는 공통적인 키워드가 들어가 있을 것이기에, 그 공통적인 키워드의 빈출을 살리기 위해서는 문서별 특이어를 잡아주는 TF-IDF보다는 공통어에 치중된 CountVectorzier의 성능이 더 좋을 것이다.

In [18]:
# 결과 확인을 위한 작업
def submission(submission_file_path,model,test_vectors):
    sample_submission = pd.read_csv(submission_file_path)
    sample_submission["target"] = model.predict(test_vectors)
    sample_submission.to_csv("submission.csv", index=False)

In [19]:
submission_file_path = 'C:/Users/USER/Desktop/Data_Practices/Disaster_Tweeter/sample_submission.csv'
test_vectors=test_vec
model = clf_nb
model.fit(train_vec, train_df['target'])
submission(submission_file_path,clf_nb,test_vectors)

위의 모델은 0.79436이라는 결과를 얻었다. 이제 조금 더 좋은 결과를 얻기 위해 개선 작업을 진행하고자 한다.

### 추가 변수 탐색

In [20]:
# keyword가 있는 경우, 이미 keyword가 text에 포함되어 있음을 확인해 볼 수 있다.
train_df.loc[train_df.keyword != '.']

,id,keyword,location,text,target
31,48,ablaze,Birmingham,bbcmtd wholesale market ablaze,1
32,49,ablaze,Est. September 2012 - Bristol,we always try to bring the heavy metal rt,0
33,50,ablaze,AFRICA,africanbaze breaking newsnigeria flag set abla...,1
34,52,ablaze,"Philadelphia, PA",cry out for more set me ablaze,0
35,53,ablaze,"London, UK",on plus side look at the sky last night it wa ...,0
...,...,...,...,...,...
7578,10830,wrecked,.,cameronhacker and i wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",three day off from work and theyve pretty much...,0
7580,10832,wrecked,London,fx forex trading cramer igers word that wrecke...,0
7581,10833,wrecked,Lincoln,engineshed great atmosphere at the british lio...,0


### 다른 Word-Embedding : Word2Vec

위의 두 모델은 단일 단어에 대해서만 고려하는 모델이다. 단어의 문맥을 고려하기 위해서는 Word2Vec이 필요하다.

In [21]:
!pip install gensim

Word2Vec을 위해서는 전체 코퍼스를 만들어야 하며,
또한 이를 위해서는 앞에처럼 CountVectorizer 단계에서 stopwords를 지정하는 것이 아닌, 사전에 이를 지정해서 제외해야 하고, 이를 위해서는 토큰화된 상황으로 시작해야한다.

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords

def text_preprocess(text):
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    tokens = tokenizer.tokenize(text)
    lemmatizes = [lemmatizer.lemmatize(token) for token in tokens]
    words = [w for w in lemmatizes if w not in stopwords.words('english')]    
    combined_word = ' '.join(w for w in words)
    
    return combined_word

In [24]:
train_df.text = train_df.text.apply(lambda s : text_preprocess(s))
test_df.text = test_df.text.apply(lambda s : text_preprocess(s))

In [25]:
# corpus 형성, 이 때 train, test 모두 해야 사이즈가 맞는다.
corpus = pd.concat([train_df, test_df]).text.values

In [26]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
corpus_token = [nltk.word_tokenize(word) for word in corpus]

In [28]:
from gensim.models import Word2Vec
word_vector_model = Word2Vec(sentences = corpus_token, vector_size = 300, 
                             window = 5, min_count = 5, sg = 1)

In [29]:
word_vector_model.wv.most_similar('help')

[('please', 0.9871126413345337),
 ('bush', 0.9855107069015503),
 ('alarm', 0.9790080189704895),
 ('could', 0.9783087968826294),
 ('drought', 0.9774770140647888),
 ('huge', 0.9754066467285156),
 ('danger', 0.9749287962913513),
 ('rage', 0.9747996926307678),
 ('evacuate', 0.9747645258903503),
 ('acre', 0.9740025401115417)]

In [30]:
word_vector_model.wv.most_similar('fire')

[('wild', 0.9464491605758667),
 ('forest', 0.9394634962081909),
 ('annihilation', 0.9214696288108826),
 ('river', 0.9198334217071533),
 ('salt', 0.9186375141143799),
 ('tonto', 0.9073498249053955),
 ('national', 0.9071736335754395),
 ('service', 0.9027889370918274),
 ('park', 0.8991431593894958),
 ('stop', 0.8965114951133728)]

Word2Vec를 통해서는 유사한 단어를 추출해준다. 다시 말해, 단어 별로 임베딩 벡터가 주어지게 된다.

## Word2Vec을 활용한 모델(RNN 비사용)

In [32]:
train_df

,id,keyword,location,text,target
0,1,.,.,deed reason earthquake may allah forgive u,1
1,4,.,.,forest fire near la ronge sask canada,1
2,5,.,.,resident asked shelter place notified officer ...,1
3,6,.,.,people receive wildfire evacuation order calif...,1
4,7,.,.,got sent photo ruby alaska smoke wildfire pour...,1
...,...,...,...,...,...
7608,10869,.,.,two giant crane holding bridge collapse nearby...,1
7609,10870,.,.,ariaahrary thetawniest control wild fire calif...,1
7610,10871,.,.,volcano hawaii,1
7611,10872,.,.,police investigating ebike collided car little...,1


In [33]:
def text_tokenize(text):
    tokenizer = nltk.tokenize.WhitespaceTokenizer()   
    tokens = tokenizer.tokenize(text)

    return tokens

In [34]:
train_df.text = train_df.text.apply(lambda s : text_tokenize(s))

In [87]:
# Disaster과의 유사성
def get_average_word2vec(tokens):
    sim_list = []
    for token in tokens:
        try:
            # 앞에까지의 문맥을 바탕으로 한 재난 연관도와, 이어옴
            sim_list.append(word_vector_model.wv.similarity(token, 'disaster'))
        except KeyError:
            continue
    if len(sim_list) == 0:
        return 0
    else:
        if len(sim_list) <= 3:
            return sum(sim_list)/len(sim_list)
        else:
            sorted_sim = sorted(sim_list, reverse = True)[:3]
            return sum(sorted_sim)/3
            

In [77]:
train_df['avg_word2vec'] = train_df.text.apply(lambda s : get_average_word2vec(s))

In [78]:
train_df

,id,keyword,location,text,target,avg_word2vec
0,1,.,.,"[deed, reason, earthquake, may, allah, forgive...",1,0.542853
1,4,.,.,"[forest, fire, near, la, ronge, sask, canada]",1,0.493892
2,5,.,.,"[resident, asked, shelter, place, notified, of...",1,0.541197
3,6,.,.,"[people, receive, wildfire, evacuation, order,...",1,0.529993
4,7,.,.,"[got, sent, photo, ruby, alaska, smoke, wildfi...",1,0.534110
...,...,...,...,...,...,...
7608,10869,.,.,"[two, giant, crane, holding, bridge, collapse,...",1,0.623134
7609,10870,.,.,"[ariaahrary, thetawniest, control, wild, fire,...",1,0.613071
7610,10871,.,.,"[volcano, hawaii]",1,0.541911
7611,10872,.,.,"[police, investigating, ebike, collided, car, ...",1,0.541644


In [79]:
train_df.loc[train_df.target == 1].describe()

,id,target,avg_word2vec
count,3271.000000,3271.0,3271.000000
mean,5661.608071,1.0,0.535080
std,3097.094809,0.0,0.083847
min,1.000000,1.0,-0.047595
25%,3104.500000,1.0,0.511653
50%,5676.000000,1.0,0.532719
75%,8252.000000,1.0,0.551172
max,10873.000000,1.0,0.993304


In [80]:
train_df.loc[train_df.target == 0].describe()

,id,target,avg_word2vec
count,4342.000000,4342.0,4342.000000
mean,5276.446338,0.0,0.518023
std,3157.206802,0.0,0.062311
min,23.000000,0.0,-0.047595
25%,2513.250000,0.0,0.501860
50%,5243.500000,0.0,0.524978
75%,8038.500000,0.0,0.540616
max,10848.000000,0.0,0.962967


In [81]:
test_df['avg_word2vec'] = test_df.text.apply(lambda s : get_average_word2vec(s))

In [82]:
X_train = train_df[['avg_word2vec']]
y_train = train_df['target']
X_test = test_df[['avg_word2vec']]
y_test = test_df['avg_word2vec']

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.3, random_state = 42,
                                                     stratify = y_train)

In [84]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_valid)

In [85]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pred, y_valid)

array([[1280,  916],
       [  23,   65]], dtype=int64)

보다시피, 음성 모델에 대해 잘 분류하고 있지 못함을 알 수 있다.
이는 단순히 주제와의 유사도만으로는 풀기에 좋지 않음을 확인했다.

In [88]:
train_df

,id,keyword,location,text,target,avg_word2vec
0,1,.,.,"[deed, reason, earthquake, may, allah, forgive...",1,0.542853
1,4,.,.,"[forest, fire, near, la, ronge, sask, canada]",1,0.493892
2,5,.,.,"[resident, asked, shelter, place, notified, of...",1,0.541197
3,6,.,.,"[people, receive, wildfire, evacuation, order,...",1,0.529993
4,7,.,.,"[got, sent, photo, ruby, alaska, smoke, wildfi...",1,0.534110
...,...,...,...,...,...,...
7608,10869,.,.,"[two, giant, crane, holding, bridge, collapse,...",1,0.623134
7609,10870,.,.,"[ariaahrary, thetawniest, control, wild, fire,...",1,0.613071
7610,10871,.,.,"[volcano, hawaii]",1,0.541911
7611,10872,.,.,"[police, investigating, ebike, collided, car, ...",1,0.541644


In [96]:
# 이 부분에 대한 개선이 필요.
def return_avg_vec(tokens):
    vec = [0.0] * 300
    count = 0
    for token in tokens:
        try:
            vec += word_vector_model.wv[token]
            count += 1
        except KeyError:
            count += 0
    if count == 0:
        return [0.0] * 300
    else:
        return [elt/count for elt in vec]

In [97]:
train_df['w2v_vector'] = train_df.text.apply(lambda s : return_avg_vec(s))
test_df['w2v_vector'] = train_df.text.apply(lambda s : return_avg_vec(s))

In [113]:
X_train = train_df[['w2v_vector']]
y_train = train_df['target']

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2,
                                                     random_state = 42, stratify = y_train)

In [127]:
X_test = test_df[['w2v_vector']]

### 신경망 적용

In [144]:
import torch
import torch.nn
tx_train = torch.Tensor(list(X_train.values))
ty_train = torch.Tensor(y_train)
tx_valid = torch.Tensor(list(X_valid.values))
ty_valid = torch.Tensor(list(y_valid))
tx_test = torch.Tensor(list(X_test.values))

torch를 바탕으로 로지스틱 회귀 문제를 풀고자 한다.

In [129]:
tx_train.shape

torch.Size([6090, 1, 300])

In [130]:
ty_train.shape

torch.Size([6090])

In [131]:
tx_test.shape

torch.Size([3263, 1, 300])

In [145]:
tx_valid.shape

torch.Size([1523, 1, 300])

In [146]:
ty_valid.shape

torch.Size([1523])

In [147]:
tx_train = tx_train.view(6090, 300)
ty_train = ty_train.view(6090, 1)
tx_test = tx_test.view(3263, 300)
tx_valid = tx_valid.view(1523, 300)
ty_valid = ty_valid.view(1523, 1)

In [148]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [149]:
# 300차원을 1차원으로 줄여야 할 것이다.

In [150]:
class DisasterClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(300, 60),
            torch.nn.ReLU())
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(60, 10),
            torch.nn.ReLU())
        self.layer3 = torch.nn.Sequential(
            torch.nn.Linear(10, 1),
            torch.nn.Sigmoid())
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        return out

In [151]:
model = DisasterClassifier()

In [157]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-7)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(tx_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, ty_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.35]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  100/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  200/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  300/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  400/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  500/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  600/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  700/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  800/1000 Cost: 0.509921 Accuracy 0.00%
Epoch  900/1000 Cost: 0.509921 Accuracy 0.00%
Epoch 1000/1000 Cost: 0.509921 Accuracy 0.00%
